<a href="https://colab.research.google.com/github/jhengineer/DeepLearning/blob/main/crossvalidation_gridsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np


# 목표: red 인지 white인지 구분하는것 (이진분류)
wine = pd.read_csv('https://bit.ly/wine_csv_data')

wine.info()
wine.describe()

wine_input = wine[['alcohol', 'sugar', 'pH']].to_numpy()
wine_target = wine['class'].to_numpy()

ss = StandardScaler()
wine_input = ss.fit_transform(wine_input)

#검증세트 구성, 60% 훈료 20% validation set 20% testset.
subtrain_input, test_input, subtrain_target, test_target = train_test_split(wine_input, wine_target, test_size = 0.2, random_state=42)
train_input, veri_input, train_target, veri_target = train_test_split(subtrain_input, subtrain_target, test_size = 0.2, random_state=42)

dt = DecisionTreeClassifier(random_state=42)
dt.fit(train_input, train_target)

print(dt.score(train_input, train_target))
print(dt.score(veri_input, veri_target))


In [ ]:
from sklearn.model_selection import cross_validate

score = cross_validate(dt, train_input, train_target)
print(score)
print(np.mean(score['test_score']))

{'fit_time': array([0.00795341, 0.00721645, 0.00705671, 0.00714803, 0.00712442]), 'score_time': array([0.00108361, 0.00105381, 0.0009737 , 0.00104165, 0.00093412]), 'test_score': array([0.84855769, 0.85096154, 0.85920578, 0.81588448, 0.84957882])}
0.8448376608349533


cross_validate 함수의 출력 결과는 다음과 같습니다:

fit_time: 각 폴드에서 모델을 훈련하는 데 걸린 시간 (초) 입니다.
score_time: 각 폴드에서 모델의 점수를 계산하는 데 걸린 시간 (초) 입니다.
test_score: 각 폴드에서 테스트 세트에 대한 모델의 점수 (정확도) 입니다.
출력 결과 아래의 0.8448376608349533는 test_score 배열에 있는 5개의 점수의 평균값입니다. 이는 교차 검증을 통해 얻은 모델의 평균 정확도를 나타냅니다.


score time이란것은.. validation fold로 계산하는거지?
네, 맞습니다. score_time은 cross_validate에서 각 폴드의 테스트 폴드에 대해 모델의 점수(이 경우 정확도)를 계산하는 데 걸린 시간을 나타냅니다. 훈련 폴드가 아닌 테스트 폴드로 계산하는 것이 맞습니다.

In [ ]:
# cross validation using splitter
# 1. StratifiedGroupKFold - class 비율 유지. 분류문제일떄 사용
# 2. KFold - 특정 비율로 그냥 나눠, Regression (수치) 일떄 쓰다.


from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold()) #default = 5
print(np.mean(scores['test_score']))

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # default 값 변경 shuffle 추가.
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8448376608349533
0.848214782205746


In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# n_job 은 cpu 사용개수 -1이면 다 써라. 디폴트가 1 한개,
gs.fit(train_input, train_target) # 여러개 교차검증해서..

dt = gs.best_estimator_
print(dt.score(train_input, train_target)) # 가장 높은 param으로 학습된 스코어.과적합 체크나 디버깅용으로만 필요. train 전체 data로 score를 매긴다.test set이 아니고.
# 이 점수는 GridSearchCV 평균 score값보다 높드라. 그래서 왜 높은지 찾아보니.. 학습된 전체데이터로 스코어를 매기다보니 당연히 높데..
print( gs.cv_results_)

print(gs.best_params_)
print(gs.cv_results_['mean_test_score'])

0.9020928554245851
{'mean_fit_time': array([0.01010575, 0.00950799, 0.00876212, 0.00818882, 0.00685101]), 'std_fit_time': array([0.00118153, 0.00079081, 0.00075092, 0.00038307, 0.00011001]), 'mean_score_time': array([0.00170484, 0.00140843, 0.00132179, 0.00127735, 0.00138474]), 'std_score_time': array([3.44658366e-04, 6.36352721e-05, 5.08863672e-05, 4.69980050e-05,
       4.86455705e-05]), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
             mask=[False, False, False, False, False],
       fill_value=1e+20), 'params': [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}], 'split0_test_score': array([0.83894231, 0.85456731, 0.85697115, 0.86177885, 0.859375  ]), 'split1_test_score': array([0.86177885, 0.86899038, 0.86538462, 0.87019231, 0.87259615]), 'split2_test_score': array([0.86040915, 0.85920578, 0.85679904, 0.85

트징	Grid Search	                     Random Search
탐색 방식	모든 조합 시도	              일부 조합 무작위 선택
속도	느림 (조합 많으면 계산 비용 ↑)	   빠름, 계산 비용 조절 가능
장점	최적 조합 보장 가능	            큰 파라미터 공간에서도 효율적
단점	파라미터 후보 많으면 비효율	      최적 조합 놓칠 가능성 있음

In [ ]:
# random search
from scipy.stats import uniform, randint # uniform 균등 난수, randint 지정값

rgen = randint(0,10) #0-9
rgen.rvs(10) # rvs=뽑아라.. 10개의 값을

array([9, 3, 9, 6, 2, 0, 0, 2, 3, 0])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True) # 1000개를 뽑아라.0이 107개 1이 99개.... 다 더하면 1000개 된다.

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([107,  99,  94,  86,  88,  99, 111,  96, 106, 114]))

In [ ]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.55861385, 0.51444491, 0.92785318, 0.51439024, 0.76592538,
       0.59021252, 0.29500094, 0.66854397, 0.02661852, 0.90331626])

In [ ]:
# 위와 같이 random을 써서 search하는게..
from sklearn.model_selection import RandomizedSearchCV
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1,25)}

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print(gs.best_params_)

print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target)) #실제 성능


{'max_depth': 34, 'min_impurity_decrease': np.float64(0.000556069984217036), 'min_samples_leaf': 15, 'min_samples_split': 20}
0.8669701124687587
0.8607692307692307
